Building Naive Bayes classifier

In [2]:
import numpy as np

class NaiveBayes:
    def __init__(self):
        self.unique_classes = None
        self.priors = None
        self.means = None
        self.stds = None
        
    def fit(self, X, y):
        """Trains model on data. Calculates means, variances, and priors for each class"""
        self.unique_classes = np.unique(y)
        num_classes = len(self.unique_classes)
        num_features = X.shape[1]

        self.means = np.zeros((num_classes, num_features))
        self.variances = np.zeros((num_classes, num_features))
        for idx, c in enumerate(self.unique_classes):
            X_c = X[y == c]  # All reviews of class c
            self.means[idx] = X_c.mean(axis=0) # Calculates the mean of the column
            self.variances = X_c.var(axis=0)
            
        self.priors = self.calculate_prior_probs(y)
            

    def calculate_likelihood((self, x, mean, var) -> float:
        """Calculates the Gaussian likelihood of the data with the given mean and variance"""
        
        
        

    def predict(self, X):
        """Performs inference using Bayes' Theorem: P(A | B) = P(B | A) * P(A) / P(B)"""
        pass

    def calculate_prior_probs(self, y):
        """Calculate proportion of words taken up by each class"""
        unique, counts = np.unique(y, return_counts=True)
        total_samples = len(y)
        num_labels = len(unique)
        prior_probs = np.zeros(n_classes)
        for idx, c in enumerate(unique):
            prior_probs[i] = counts[idx] / total_samples
        return prior_probs
        
        

SyntaxError: unterminated string literal (detected at line 33) (4270150086.py, line 33)

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Sample reviews
reviews = [
    "This movie is great",
    "I love this movie",
    "This movie is terrible"
]

# Create and fit the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(reviews)

# Convert to matrix
matrix = tokenizer.texts_to_matrix(reviews, mode='freq')

print("Vocabulary:", tokenizer.word_index)
print("Matrix shape:", matrix.shape)
print("Matrix:")
print(matrix)


Vocabulary: {'this': 1, 'movie': 2, 'is': 3, 'great': 4, 'i': 5, 'love': 6, 'terrible': 7}
Matrix shape: (3, 8)
Matrix:
[[0.   0.25 0.25 0.25 0.25 0.   0.   0.  ]
 [0.   0.25 0.25 0.   0.   0.25 0.25 0.  ]
 [0.   0.25 0.25 0.25 0.   0.   0.   0.25]]
